In [1]:
import kagglehub
kagglehub.login()

In [2]:
trec_covid_information_retrieval_path = kagglehub.competition_download('trec-covid-information-retrieval')


In [3]:
# Check xem đã tải thành công chưa

import os

path = "/kaggle/input/trec-covid-information-retrieval"
print(os.listdir(path))


['sample_submission.csv', 'topics-rnd3.csv', 'CORD-19', 'docids-rnd3.txt', 'qrels.csv']


In [43]:
cord19_path = "/kaggle/input/trec-covid-information-retrieval/CORD-19/CORD-19"
print(os.listdir(new_path))


['cord_19_embeddings_2020-05-19.csv', 'document_parses', 'COVID.DATA.LIC.AGMT.pdf', 'metadata.csv']


****Import****

In [7]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import csv
import os
import json
from collections import defaultdict

****Preprocessing data****

In [49]:
metadata_path = "/kaggle/input/trec-covid-information-retrieval/CORD-19/CORD-19/metadata.csv"
dataset_root = "/kaggle/input/trec-covid-information-retrieval/CORD-19/CORD-19"

cord_uid_to_text = defaultdict(list)

# open the file
with open(metadata_path) as f_in:
    reader = csv.DictReader(f_in)
    for row in reader:
        # access some metadata
        cord_uid = row['cord_uid']
        title = row['title']
        abstract = row['abstract']
        authors = row['authors'].split('; ')        

        # access the full text (if available) for Intro
        introduction = []
        if row['pdf_json_files']:
            for json_path in row['pdf_json_files'].split('; '):
                full_json_path = os.path.join(dataset_root, json_path)
                with open(full_json_path) as f_json:
                    full_text_dict = json.load(f_json)

                    # grab introduction section from *some* version of the full text
                    for paragraph_dict in full_text_dict['body_text']:
                        paragraph_text = paragraph_dict['text']
                        section_name = paragraph_dict['section']
                        if 'intro' in section_name.lower():
                            introduction.append(paragraph_text)             

                    # stop searching other copies of full text if already got introduction
                    if introduction:
                        break
                

        # save for later usage
        cord_uid_to_text[cord_uid].append({
            'title': title,
            'abstract': abstract,
            'introduction': introduction
        })


**Chunking Dataset**

In [ ]:
# MARK: Constant
max_words_chunking = 300
some_cord_uid = list(cord_uid_to_text.keys())[0]
paragraphs = cord_uid_to_text[some_cord_uid][0]['introduction']
text = " ".join(paragraphs)


def chunking(paragraph, max_words_chunking = max_words_chunking):
    words = paragraph.split() 
    chunks = []
    for i in range(0, len(words), max_words_chunking):
        chunk = " ".join(words[i:i+max_words_chunking])
        chunks.append(chunk)
    return chunks
        
chunk = chunking(text, max_words_chunking)

print("chunk", chunk)

chunk ['Mycoplasma pneumoniae is a common cause of upper and lower respiratory tract infections. It remains one of the most frequent causes of atypical pneumonia particu-larly among young adults. [1, 2, 3, 4, 5] Although it is highly transmissible, most infections caused by this organism are relatively minor and include pharyngitis, tracheobronchitis, bronchiolitis, and croup with one fifth of in-fections being asymptomatic. [6, 7] Only 3 -10% of infected subjects develop symptoms consistent with bronchopneumonia and mortality from infection is rare. [6, 7] The organism is fastidious and difficult to grow on cultures. Therefore, diagnosis of infections caused by this organism is usually confirmed with serological tests or polymerase chain reaction-gene amplification techniques. At King Abdulaziz University Hospital (KAUH), Jeddah, Saudi Arabia, the facility to perform Mycoplasma culture has been available since January 1997. As published information concerning M. pneumoniae infections 